<mark>DATA ENGINEER

============= Importar librerias

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

=============== Cargar dataset 'credits'

In [3]:
data_credits = pd.read_csv('credits.csv')
data_credits

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862
...,...,...,...
45471,"[{'cast_id': 0, 'character': '', 'credit_id': ...","[{'credit_id': '5894a97d925141426c00818c', 'de...",439050
45472,"[{'cast_id': 1002, 'character': 'Sister Angela...","[{'credit_id': '52fe4af1c3a36847f81e9b15', 'de...",111109
45473,"[{'cast_id': 6, 'character': 'Emily Shaw', 'cr...","[{'credit_id': '52fe4776c3a368484e0c8387', 'de...",67758
45474,"[{'cast_id': 2, 'character': '', 'credit_id': ...","[{'credit_id': '533bccebc3a36844cf0011a7', 'de...",227506


=============== Cargar dataset 'movies'

In [4]:
data_movies = pd.read_csv('movies_dataset.csv', encoding='UTF-8')
pd.options.display.max_columns=0
data_movies.head(2)

C:\Users\M&A\AppData\Local\Temp\ipykernel_16364\3241442492.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  data_movies = pd.read_csv('movies_dataset.csv', encoding='UTF-8')


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [5]:
print(f'El dataframe tiene: {data_movies.shape[1]} columnas y {data_movies.shape[0]} filas \n')
print(f'COLUMNA --------------- TIPO DE DATO\n{data_movies.dtypes}')

El dataframe tiene: 24 columnas y 45466 filas 

COLUMNA --------------- TIPO DE DATO
adult                     object
belongs_to_collection     object
budget                    object
genres                    object
homepage                  object
id                        object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity                object
poster_path               object
production_companies      object
production_countries      object
release_date              object
revenue                  float64
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
video                     object
vote_average             float64
vote_count               float64
dtype: object


==================== Desanidar columnas del dataset 'movies'

================ belongs_to_collection

In [6]:
df_belongs_to_collection = data_movies['belongs_to_collection'].str.lstrip('{').str.rstrip('}').str.replace("'", '')
df_belongs_to_collection = df_belongs_to_collection.str.replace("id: ", '')
df_belongs_to_collection = df_belongs_to_collection.str.replace(" name: ", '')
df_belongs_to_collection = df_belongs_to_collection.str.replace("poster_path: ", '')
df_belongs_to_collection = df_belongs_to_collection.str.replace("backdrop_path: ", '')
df_belongs_to_collection = df_belongs_to_collection.str.split(pat =',', expand = True)
df_belongs_to_collection = df_belongs_to_collection.rename(columns = {0:'id_belongs',
                                                                        1:'name_belongs',
                                                                        2:'poster_path_belongs',
                                                                        3:'backdrop_path_belongs'}).iloc[:,0:4]
df_belongs_to_collection

,id_belongs,name_belongs,poster_path_belongs,backdrop_path_belongs
0,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg
1,NaN,NaN,NaN,NaN
2,119050,Grumpy Old Men Collection,/nLvUdqgPgm3F85NMCii9gVFUcet.jpg,/hypTnLot2z8wpFS7qwsQHW1uV8u.jpg
3,NaN,NaN,NaN,NaN
4,96871,Father of the Bride Collection,/nts4iOmNnq7GNicycMJ9pSAn204.jpg,/7qwE57OVZmMJChBpLEbJEmzUydk.jpg
...,...,...,...,...
45461,NaN,NaN,NaN,NaN
45462,NaN,NaN,NaN,NaN
45463,NaN,NaN,NaN,NaN
45464,NaN,NaN,NaN,NaN


================ genres

In [7]:
df_genres = data_movies['genres'].str.replace(r"[0-9 ':[\]{}]", "", regex=True).str.replace('id,name', '')
df_genres = df_genres.str.split(',')
df_genres


0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45461                 [Drama, Family]
45462                         [Drama]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genres, Length: 45466, dtype: object

========================= production_companies

In [8]:
df_production_companies = data_movies['production_companies'].str.replace(r"[0-9':[\]{}]", "", regex=True)
df_production_companies = df_production_companies.str.replace('name', '').str.replace(', id', '')
df_production_companies = df_production_companies.str.split(',')
for ipc in range(len(df_production_companies)):
    if type(df_production_companies[ipc]) == list:
        for npc in range(len(df_production_companies[ipc])):
            df_production_companies[ipc][npc] = df_production_companies[ipc][npc].strip()
df_production_companies

0                                [Pixar Animation Studios]
1        [TriStar Pictures, Teitler Film, Interscope Co...
2                           [Warner Bros., Lancaster Gate]
3                 [Twentieth Century Fox Film Corporation]
4             [Sandollar Productions, Touchstone Pictures]
                               ...                        
45461                                                   []
45462                                        [Sine Olivia]
45463                            [American World Pictures]
45464                                          [Yermoliev]
45465                                                   []
Name: production_companies, Length: 45466, dtype: object

========================= production_countries

In [9]:
df_production_countries = data_movies['production_countries'].str.replace(r"[0-9':[\]{}]", "", regex=True)
df_production_countries = df_production_countries.str.replace('name', '').str.replace('iso__', '')
df_production_countries = df_production_countries.str.split(',')

for ipco in range(len(df_production_countries)):
    if type(df_production_countries[ipco]) == list:
        for npco in range(len(df_production_countries[ipco])):
            df_production_countries[ipco][npco] = df_production_countries[ipco][npco].strip()
            
for ipco1 in range(len(df_production_countries)):
    lista_df_production_countries = []
    if type(df_production_countries[ipco1]) == list:
        for npco1 in range(len(df_production_countries[ipco1])):
            if npco1%2 != 0:
                lista_df_production_countries.append(df_production_countries[ipco1][npco1])
        df_production_countries[ipco1] = lista_df_production_countries[::]
df_production_countries

0        [United States of America]
1        [United States of America]
2        [United States of America]
3        [United States of America]
4        [United States of America]
                    ...            
45461                        [Iran]
45462                 [Philippines]
45463    [United States of America]
45464                      [Russia]
45465              [United Kingdom]
Name: production_countries, Length: 45466, dtype: object

========================= spoken_languages

In [10]:
df_spoken_languages = data_movies['spoken_languages'].str.replace(r"[0-9':[\]{}]", "", regex=True)
df_spoken_languages = df_spoken_languages.str.replace('name', '').str.replace('iso__', '')
df_spoken_languages = df_spoken_languages.str.split(',')

for isl in range(len(df_spoken_languages)):
    if type(df_spoken_languages[isl]) == list:
        for nsl in range(len(df_spoken_languages[isl])):
            df_spoken_languages[isl][nsl] = df_spoken_languages[isl][nsl].strip()
    else:
        df_spoken_languages[isl] = df_spoken_languages[isl]

for isl1 in range(len(df_spoken_languages)):
    lista_df_spoken_languages = []
    if type(df_spoken_languages[isl1]) == list:
        for nsl1 in range(len(df_spoken_languages[isl1])):
            if nsl1%2 == 0:
                lista_df_spoken_languages.append(df_spoken_languages[isl1][nsl1])
        df_spoken_languages[isl1] = lista_df_spoken_languages[::]
df_spoken_languages

0            [en]
1        [en, fr]
2            [en]
3            [en]
4            [en]
           ...   
45461        [fa]
45462        [tl]
45463        [en]
45464          []
45465        [en]
Name: spoken_languages, Length: 45466, dtype: object

================= Union de los dataframes

In [11]:
data_movies = data_movies.loc[:,['adult',
                                 'budget', 
                                 'homepage',
                                 'id', 
                                 'imdb_id', 
                                 'original_language', 
                                 'original_title', 
                                 'overview', 
                                 'popularity', 
                                 'poster_path', 
                                 'release_date', 
                                 'revenue', 
                                 'runtime',
                                 'status', 
                                 'tagline', 
                                 'title', 
                                 'video', 
                                 'vote_average', 
                                 'vote_count']]
data_movies.head(2)

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [12]:
data_movies = pd.concat([data_movies,
                         df_belongs_to_collection,
                         df_genres,
                         df_production_companies,
                         df_production_countries,
                         df_spoken_languages], axis=1)
data_movies.head(2)

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,id_belongs,name_belongs,poster_path_belongs,backdrop_path_belongs,genres,production_companies,production_countries,spoken_languages
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"[Animation, Comedy, Family]",[Pixar Animation Studios],[United States of America],[en]
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN,NaN,NaN,"[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],"[en, fr]"


==================== revenue, budget deben ser rellenados por el número 0

In [13]:
data_movies['revenue'] = data_movies['revenue'].fillna(0)
data_movies['budget'] = data_movies['budget'].fillna(0)
data_movies.head(2)

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,id_belongs,name_belongs,poster_path_belongs,backdrop_path_belongs,genres,production_companies,production_countries,spoken_languages
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"[Animation, Comedy, Family]",[Pixar Animation Studios],[United States of America],[en]
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN,NaN,NaN,"[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],"[en, fr]"


====================== Los valores nulos del campo 'release_date' deben eliminarse.

In [14]:
data_movies = data_movies.dropna(subset=['release_date'])
data_movies.reset_index(inplace=True, drop=True)
data_movies.head(2)

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,id_belongs,name_belongs,poster_path_belongs,backdrop_path_belongs,genres,production_companies,production_countries,spoken_languages
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"[Animation, Comedy, Family]",[Pixar Animation Studios],[United States of America],[en]
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN,NaN,NaN,"[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],"[en, fr]"


================== De haber fechas, deberán tener el formato AAAA-mm-dd, además deberán crear la columna release_year donde extraerán el año de la fecha de estreno.

In [15]:
data_movies['release_date'] = data_movies['release_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d').date() if len(x) == 10 else None)
data_movies = data_movies.dropna(subset=['release_date'])
data_movies.reset_index(inplace=True, drop=True)
data_movies['year_release_date'] = data_movies['release_date'].apply(lambda x: x.year)
data_movies.head(2)

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,id_belongs,name_belongs,poster_path_belongs,backdrop_path_belongs,genres,production_companies,production_countries,spoken_languages,year_release_date
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"[Animation, Comedy, Family]",[Pixar Animation Studios],[United States of America],[en],1995
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN,NaN,NaN,"[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],"[en, fr]",1995


========== Crear la columna con el retorno de inversión, llamada return con los campos revenue y budget, dividiendo estas dos últimas revenue / budget, cuando no hay datos disponibles para calcularlo, deberá tomar el valor 0.

In [16]:
data_movies['return'] = data_movies['revenue']/data_movies['budget'].astype('float')
data_movies['return'] = data_movies['return'].apply(lambda x: 0 if np.isnan(x) or np.isinf(x) else x)
data_movies.head(2)

,adult,budget,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,release_date,revenue,runtime,status,tagline,title,video,vote_average,vote_count,id_belongs,name_belongs,poster_path_belongs,backdrop_path_belongs,genres,production_companies,production_countries,spoken_languages,year_release_date,return
0,False,30000000,http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,1995-10-30,373554033.0,81.0,Released,NaN,Toy Story,False,7.7,5415.0,10194,Toy Story Collection,/7G9915LfUQ2lVfwMEEhDsn3kT4B.jpg,/9FBwqcd9IRruEDUrTdcaafOMKUq.jpg,"[Animation, Comedy, Family]",[Pixar Animation Studios],[United States of America],[en],1995,12.451801
1,False,65000000,NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0,NaN,NaN,NaN,NaN,"[Adventure, Fantasy, Family]","[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],"[en, fr]",1995,4.043035


================ Eliminar las columnas que no serán utilizadas: video, imdb_id, adult, original_title, poster_path y homepage.

In [17]:
data_movies = data_movies.loc[:,['id',
                                 'title',
                                 'name_belongs',
                                 'overview',
                                 'genres',
                                 'original_language',
                                 'spoken_languages',
                                 'popularity',
                                 'release_date',
                                 'year_release_date',
                                 'production_companies',
                                 'production_countries',
                                 'runtime',
                                 'status',
                                 'tagline',
                                 'vote_average',
                                 'vote_count',
                                 'budget',
                                 'revenue', 
                                 'return']]
data_movies.head(2)

,id,title,name_belongs,overview,genres,original_language,spoken_languages,popularity,release_date,year_release_date,production_companies,production_countries,runtime,status,tagline,vote_average,vote_count,budget,revenue,return
0,862,Toy Story,Toy Story Collection,"Led by Woody, Andy's toys live happily in his ...","[Animation, Comedy, Family]",en,[en],21.946943,1995-10-30,1995,[Pixar Animation Studios],[United States of America],81.0,Released,NaN,7.7,5415.0,30000000,373554033.0,12.451801
1,8844,Jumanji,NaN,When siblings Judy and Peter discover an encha...,"[Adventure, Fantasy, Family]",en,"[en, fr]",17.015539,1995-12-15,1995,"[TriStar Pictures, Teitler Film, Interscope Co...",[United States of America],104.0,Released,Roll the dice and unleash the excitement!,6.9,2413.0,65000000,262797249.0,4.043035


In [18]:
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45376 entries, 0 to 45375
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    45376 non-null  object 
 1   title                 45376 non-null  object 
 2   name_belongs          4488 non-null   object 
 3   overview              44435 non-null  object 
 4   genres                45376 non-null  object 
 5   original_language     45365 non-null  object 
 6   spoken_languages      45376 non-null  object 
 7   popularity            45376 non-null  object 
 8   release_date          45376 non-null  object 
 9   year_release_date     45376 non-null  int64  
 10  production_companies  45376 non-null  object 
 11  production_countries  45376 non-null  object 
 12  runtime               45130 non-null  float64
 13  status                45296 non-null  object 
 14  tagline               20398 non-null  object 
 15  vote_average       

In [19]:
data_movies = data_movies.astype({'id':'int', 'popularity':'float', 'budget':'float'})

In [20]:
data_movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45376 entries, 0 to 45375
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    45376 non-null  int32  
 1   title                 45376 non-null  object 
 2   name_belongs          4488 non-null   object 
 3   overview              44435 non-null  object 
 4   genres                45376 non-null  object 
 5   original_language     45365 non-null  object 
 6   spoken_languages      45376 non-null  object 
 7   popularity            45376 non-null  float64
 8   release_date          45376 non-null  object 
 9   year_release_date     45376 non-null  int64  
 10  production_companies  45376 non-null  object 
 11  production_countries  45376 non-null  object 
 12  runtime               45130 non-null  float64
 13  status                45296 non-null  object 
 14  tagline               20398 non-null  object 
 15  vote_average       

In [24]:
data_movies.to_csv('data_movies.csv')

<mark>DESARROLLO API

In [22]:
from fastapi import FastAPI

In [23]:
app = FastAPI()

def index():
    return 'Hola'